In [33]:
import numpy as np
import cv2
import heapq

In [34]:
h = 50
w = 180

map = np.zeros((h, w, 3), dtype=np.uint8)

valid = False
while not valid:
    clearance = int(input("Enter clearance 0 - 6[mm]: "))
    radius = int(input("Enter robot radius 1 - 6[mm]: "))
    buffer = clearance+radius
    if clearance < 0 or clearance > 6:
        print("choose clearance between 0 and 6")
    elif radius < 1 or radius > 6:
        print("choos radius between 1 and 6")
    else:
        valid = True


# set boundary around border of map
map_limits = map.copy()
map_limits = cv2.cvtColor(map_limits, cv2.COLOR_BGR2GRAY)
map_limits = cv2.rectangle(map_limits, (0,0), (w,h), (255), buffer)
locations = np.where(map_limits == (255))

# initialize boundary sets and add the border boundary points
boundary = set(zip(locations[1], locations[0]))

# delete variables that are no longer used
del map_limits
del locations

# function for adding boundary points around obstacles
def add_boundary(inpt_x,inpt_y):
    # add given point to the boundary sets
    boundary.add((inpt_x,inpt_y))

    # add all points in a box around given point to boundary sets
    for x in range(inpt_x-buffer, inpt_x+(buffer+1)):
        for y in range(inpt_y-buffer, inpt_y+(buffer+1)):
            boundary.add((x,y))

# define obstacles
for y in range(17,35):
    # define obstacle E
    for x in range(20,40):
        if 16 < y <= 20:
            if 21 < x <= 39:
                # top horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if 20 < y <= 23:
            if 21 < x <= 25:
                # between top and middle horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if 23 < y <=27:
            if 21 < x <= 39:
                # middle horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if 27 < y <= 30:
            if 21 < x <= 25:
                # between middle and bottom horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if 30 < y <= 34:
            if 21 < x <= 39:
                #bottom horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)
    
    # define obstacle N
    for x in range(40,60):
        if 41 < x <= 45:
            # first vertical of N
            map[y,x] = (255,0,0)
            add_boundary(x,y)

            # second vertical of N
        if 55 < x <= 59:
            map[y,x] = (255,0,0)
            add_boundary(x,y)

            # diagonal of N determined by equations of two lines
        if 45 < x <= 55:
            if (1.2*x)-38 < y <= (1.2*x)-32:
                map[y,x] = (255,0,0)
                add_boundary(x,y)
    
    # define obstacle P
    for x in range(60, 80):

        # main vertical of P
        if 61 < x <=67:
            map[y,x] = (255,0,0)
            add_boundary(x,y)

        # round part of P determined by ellipse equation
        if 67 < x <= 79:
            if (((x-67)**2)/(12**2))+(((y-22)**2)/(6**2)) <= 1:
                map[y,x] = (255,0,0)
                add_boundary(x,y)
    
    # define obstacle M
    for x in range(80,100):

        # first vertical of M
        if 81 < x <=85:
            map[y,x] = (255,0,0)
            add_boundary(x,y)
        
        # first angle of M determined by equations of two lines
        if 85 < x <= 90:
            if ((9/5)*x)-137 < y <= ((9/5)*x)-129:
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        # second angle of M determined by equations of two lines
        if 90 < x <= 95:
            if ((-9/5)*x)+187 < y <= ((-9/5)*x)+195:
                map[y,x] = (255,0,0)
                add_boundary(x,y)
        
        # last vertical of M 
        if 95 < x <= 99:
            map[y,x] = (255,0,0)
            add_boundary(x,y)
    
    # define obstacle 6
    for x in range(100,120):

        # draw main circle per equation
        if ((x-110)**2)+(y-28.5)**2 <= 5.5**2:
            # add both 6's spaced 20 pixels apart
            map[y,x] = (255,0,0)
            map[y,x+20] = (255,0,0)
            add_boundary(x,y)
            add_boundary(x+20,y)

        # draw second circle at tip of "tail" of 6
        if ((x-110.85)**2)+(y-18)**2 <= 2**2:
            map[y,x] = (255,0,0)
            map[y,x+20] = (255,0,0)
            add_boundary(x,y)
            add_boundary(x+20,y)
        
        # draw part of a ring defined by two circles and two lines
        # to define the "tail" of 6
        if x-92.85 < y <=28.5:
            if 12.85**2 < (((x-121.35)**2)+((y-28.5)**2)) <= 16.85**2:
                map[y,x] = (255,0,0)
                map[y,x+20] = (255,0,0)
                add_boundary(x,y)
                add_boundary(x+20,y)

    # define obstacle 1
    for x in range(140,160):
        if 16 < y <= 20.62:
            if 144 < x <= 152:
                # top horizontal of 1
                map[y,x] = (255,0,0)
                add_boundary(x,y)
        if 20.62 < y <= 30:
            if 148 < x <= 152:
                # main vertical of 1
                map[y,x] = (255,0,0)
                add_boundary(x,y)
        if 30 < y <= 34:
            if 144 < x < 156:
                # bottom horizontal of 1
                map[y,x] = (255,0,0)
                add_boundary(x,y)
  

In [35]:
# define start and end points
valid = False
while not valid:
    start_x = int(input("Enter start x: "))
    start_y = 50-int(input("Enter start y: "))
    start_theta = int(input("Enter multiple of 30 for start angle [deg]: "))
    end_x = int(input("Enter goal x: "))
    end_y = 50-int(input("Enter goal y: "))
    end_theta = int(input("Enter multiple of 30 for goal angle [deg]: "))
    step_size = int(input("Enter step size from 1 to 10: "))
    
    start = (start_x, start_y,start_theta)
    end = (end_x, end_y,end_theta)
    if start_x <= 0 or start_x >= w:
        print("Start position out of bounds")
    elif start_y <= 0 or start_y >= h:
        print("Start position out of bounds")
    elif start_theta%30 != 0:
        print("Start angle must be multiple of 30")
    elif end_x <= 0 or end_x >= w:
        print("Goal position out of bounds")
    elif end_y <= 0 or end_y >= h:
        print("Goal position out of bounds")
    elif end_theta%30 != 0:
        print("Goal angle must be multiple of 30")
    elif start in boundary:
        print("Invalid start position")
    elif end in boundary:
        print("Invalid goal position")
    elif step_size < 1 or step_size > 10:
        print("Invalid step size")
    else:
        valid = True


In [36]:
# initialize lists for a star search
open_list = []
closed_list = []

# confirm open and closed lists are heapq
heapq.heapify(closed_list)
heapq.heapify(open_list)

# add start node to open list
#(total, cost to come, cost to go, parent, node)
def distance(p1,p2):
    dist = int(np.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2)))
    return dist
start_ctg = distance(start,end)
heapq.heappush(open_list, (start_ctg, 0, start_ctg, start, start))

In [37]:
def move(node,angle):
    p_ctc = node[1]
    p_x = node[4][0]
    p_y = node[4][1]
    n_x = int(p_x+(step_size*np.cos(np.deg2rad(angle))))
    n_y = int(p_y+(step_size*np.sin(np.deg2rad(angle))))
    n_t = angle
    p_coord = node[4]
    c_coord = (n_x, n_y, n_t)
    c_xy = (n_x, n_y)
    c_ctc = p_ctc+step_size
    c_ctg = distance(c_coord,end)
    c_tot = c_ctc+c_ctg
    if c_xy in boundary:
        return
    for item in open_list:
        if item[4][0] == c_coord[0] and item[4][1] == c_coord[1]:
            if item[0] > c_tot:
                open_list.remove(item)
            else:
                return
    heapq.heappush(open_list, (c_tot, c_ctc, c_ctg, p_coord, c_coord))
    return

dist_to_goal = start_ctg

while dist_to_goal > 3:
    parent_node = heapq.heappop(open_list)
    boundary.add((parent_node[4][0], parent_node[4][1]))
    heapq.heappush(closed_list, parent_node)
    dist_to_goal = distance(parent_node[4],end)
    # print(dist_to_goal)

    for item in [-2, -1, 0, 1, 2]:
        ang = parent_node[4][2] + (30*item)
        move(parent_node,ang)

  

In [38]:
final_path = []
path_node = closed_list[-1][4]
while path_node != start:
    for item in closed_list:
        if item[4] == path_node: 
            final_path.append(item)
            path_node = item[3]

final_path.sort() # reorder path to go from start to finish


In [39]:
map_display = map.copy()

for item in closed_list:
    par_xy = (item[3][0],item[3][1])
    chi_xy = (item[4][0],item[4][1])
    cv2.line(map_display,par_xy,chi_xy,(255,255,255),1)
    frame = map_display.copy()
    frame = cv2.resize(frame,(0,0), fx=3, fy=3)
    cv2.imshow('animation',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

for item in final_path:
    par_xy = (item[3][0],item[3][1])
    chi_xy = (item[4][0],item[4][1])
    cv2.line(map_display,par_xy,chi_xy,(255,0,0),1)
    frame = map_display.copy()
    frame = cv2.resize(frame,(0,0), fx=3, fy=3)
    cv2.imshow('animation',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.waitKey(0)
cv2.destroyAllWindows()